# Explore BL Sessions Data

Here we introduce some useful DataJoint tools to explore the bl19 pipeline

First thing first, import datajoint

In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()


Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [2]:
import datajoint as dj
import utility.blob_transformation as bt
import numpy as np
import pylab as plt
import matplotlib.patches as mpatches
from matplotlib import cm

import pandas as pd

In [3]:
dj.blob.use_32bit_dims = True

## Get handle to DBs

lab         =  researchers & rig DB <br />
subject     =  rats DB <br />
acquisition =  sessions DB

In [4]:
bdata = dj.create_virtual_module('bdatatest', 'bdatatest')
session_key = {'sessid': 822635}
session_data = (bdata.Sessions & session_key).fetch('protocol_data', as_dict=True)
parsed_events = (bdata.ParsedEvents & session_key).fetch(as_dict=True)

[2022-08-05 10:24:22,781][INFO]: Connecting alvaros@datajoint01.pni.princeton.edu:3306
[2022-08-05 10:24:23,111][INFO]: Connected alvaros@datajoint01.pni.princeton.edu:3306


In [6]:
protocol_data = bt.mymblob_to_dict(session_data[0]['protocol_data'])
protocol_data

{'hits': array([nan,  1.,  1.,  0.,  1.,  0., nan]),
 'sides': 'llllrrl',
 'tasks': 'ddddddd',
 'stage': 'Stage 10'}

In [8]:
peh = bt.mymblob_to_dict(parsed_events[0]['peh'])
peh.keys()

dict_keys(['pokes', 'waves', 'states'])

In [10]:
peh['states']

(array([[        nan,  883.515104],
        [1008.358804,         nan]]),
 array([[        nan,  883.515204],
        [1008.358804, 1008.358804]]),
 array([[883.515104,        nan],
        [883.515204, 884.515204]]),
 array([[884.515204, 884.515304]]),
 array([[884.515304, 885.515304]]),
 array([], shape=(0, 2), dtype=float64),
 array([[ 885.515304, 1005.235304]]),
 array([[1005.235304, 1005.236304]]),
 array([[1005.236304, 1005.236604]]),
 array([[1005.236604, 1005.257904]]),
 array([[1005.257904, 1005.357904]]),
 array([], shape=(0, 2), dtype=float64),
 array([], shape=(0, 2), dtype=float64),
 array([], shape=(0, 2), dtype=float64),
 array([], shape=(0, 2), dtype=float64),
 array([], shape=(0, 2), dtype=float64),
 array([], shape=(0, 2), dtype=float64),
 array([[1005.357904, 1005.358204]]),
 array([[1005.358204, 1008.358104]]),
 array([], shape=(0, 2), dtype=float64),
 array([[1008.358104, 1008.358804]]),
 array(['state_0'], dtype='<U7'),
 array(['state_0'], dtype='<U7'))